# Create a PISM domain from a polygon based on the BedMachine v5 grid

In [ ]:
import geopandas as gp
import xarray as xr
import numpy as np
import matplotlib.pylab as plt
from pism_ragis.domain import get_bounds, create_domain

In [ ]:
# Coordinate Reference System 
crs = "EPSG:3413"

# the base resolution of BedMachine in meters
base_resolution: int = 150

# the resolutions that you want supported:
# 150, 300, 450, 600, 900, 1200, 1500, 1800, 2400, 3000, 3600, and 4500m
multipliers = [1, 2, 3, 4, 6, 8, 10, 12, 16, 20, 24, 30]

# buffer in m
buffer = 3e3

# Path to BedMachine file, open using xarray
ds = xr.open_dataset("/Users/andy/Google Drive/My Drive/data/MCDataset/BedMachineGreenland-v5.nc")
ds = ds.rio.set_spatial_dims(x_dim="x", y_dim="y")
ds.rio.write_crs(crs, inplace=True)

In [ ]:
# Path to polygon file, open using GeoPandas
# Drop all rows that have None as the basin Name
basins = gp.read_file("/Users/andy/Google Drive/My Drive/data/Greenland_Basins_PS_v1.4.2/Greenland_Basins_PS_v1.4.2.shp").to_crs(crs).dropna(subset=["Name"])

In [ ]:
# Basic example:
# We loop over all basins in the "basins" geopandas dataframe assuming that an "Name" attribute exists.
# First we add the buffer, then we extract the bounding box, and finally we calculate the domain and 
# save it as a netCDF file.

# This should be parallelized with Dask in the future.

for m_id, basin in basins.iterrows():
    name = basin["Name"]
    print(f"Processing basin {m_id}: {name}")
    minx, miny, maxx, maxy = basin.geometry.buffer(buffer).bounds
    ll = ds.sel({"x": minx, "y": miny}, method="nearest")
    ur = ds.sel({"x": maxx, "y": maxy}, method="nearest")
    tmp_ds = ds.sel({"x": slice(ll["x"], ur["x"]), "y": slice(ur["y"], ll["y"])})
    x_bnds, y_bnds = get_bounds(tmp_ds, 
                                base_resolution=base_resolution,
                                multipliers=multipliers)
    sub_ds = ds.sel({"x": slice(*x_bnds), "y": slice(*y_bnds[::-1])})
    grid = create_domain(x_bnds, y_bnds)
    grid.attrs.update({"domain": name})
    grid.to_netcdf(f"{m_id}_{name}.nc")
    # Save a PDF
    fig = plt.figure()
    ax = fig.add_subplot(111)
    sub_ds["bed"].plot(ax=ax, cmap="gray")
    basins.iloc[[m_id]].plot(ax=ax, alpha=0.5)
    ax.set_title(name)
    fig.savefig(f"{m_id}_{name}.pdf")